# Module Alignment

Steps:
- convert found tracks to csv (there is no better way yet)
- give recos to align method below
- select correct anchor points (one set for each momentum)
- run

Command for conversion is in `src/util` and will be something like:

```bash
root -l -q 'readRootTrackRecos.C("/mnt/work/himsterData/LumiFit/plab_1.50GeV/dpm_elastic_theta_2.7-13.0mrad_recoil_corrected/ip_offset_XYZDXDYDZ_0.0_0.0_0.0_0.0_0.0_0.0/beam_grad_XYDXDY_0.0_0.0_0.0_0.0/geo_misalignmentmisMat-modules/100000/1-100_uncut/no_alignment_correction","recos-1.5-misMod.csv")' 
```


In [1]:
#! specify reco file and anchor point file here, then just run the entire notebook without a second thought like a mad man
recoFile = "../src/util/lmd-1.5-Vf.csv" # current (2024) run
# recoFile = "../src/util/lmd-15-jP.csv" # current (2024) run
anchorPointFile = "../config/anchorPoints/anchorPoints-1.5-aligned.json"
# anchorPointFile = "../config/anchorPoints/anchorPoints-15.00-aligned.json"
alignmentMatrixFilename = "../matrices/100u-case-1/moduleAlignmentMatrices-1.5.json"
# alignmentMatrixFilename = "../matrices/100u-case-1/moduleAlignmentMatrices-15.00.json"


In [2]:
import numpy as np
import uproot, json
from pathlib import Path

# from util.matrix import loadMatrices, baseTransform

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

In [3]:
def loadMatrices(filename):
    with open(filename) as f:
        result = json.load(f)
    for key, value in result.items():
        result[key] = np.array(value).reshape(4, 4)
    return result


def saveMatrices(matrices, fileName):

    # create path if needed
    if not Path(fileName).parent.exists():
        Path(fileName).parent.mkdir()

    # warn if overwriting
    if Path(fileName).exists():
        print(f"WARNING. Replacing file: {fileName}!\n")
        Path(fileName).unlink()

    # flatten matrices, make a copy (pass-by-reference!)
    saveMatrices = {}
    for p in matrices:
        saveMatrices[p] = np.ndarray.tolist(np.ndarray.flatten(matrices[p]))

    with open(fileName, "w") as f:
        json.dump(saveMatrices, f, indent=2, sort_keys=True)


def baseTransform(mat, matFromAtoB):
    """
    Reminder: the way this works is that the matrix pointing from pnd to sen0 transforms a matrix IN sen0 back to Pnd
    If you want to transform a matrix from Pnd to sen0, and you have the matrix to sen0, then you need to give
    this function inv(matTo0). I know it's confusing, but that's the way this works.

    Example: matrixInPanda = baseTransform(matrixInSensor, matrixPandaToSensor)
    """
    return matFromAtoB @ mat @ np.linalg.inv(matFromAtoB)

In [4]:
with open("../config/sensorIDtoSectorID.json", "r") as f:
    sensorIDdict = json.load(f)
    sectorIDlut = np.empty(len(sensorIDdict))

    # create a look up table for fast sensorID -> sectorID translation
    for key, value in sensorIDdict.items():
        sectorIDlut[int(key)] = value


In [5]:
def readRecoHitsFromCSVFile(filename):
    csvValues = np.genfromtxt(filename, delimiter=",")
    # check if there really are n times 4 elements
    if len(csvValues) % 4 != 0:
        raise ValueError('entries are not a multiple of 4!')
    
    nEvents = int(len(csvValues) / 4)

    ids = np.array(csvValues[:,0],dtype=int)
    csvValues[:,0] = sectorIDlut[ids]

    csvValues = csvValues.reshape((nEvents,4,4))

    return csvValues


In [6]:
def readRecoHitsFromRootFiles(filename, maxNoOfFiles=0):
    """
    This funtion currently doesn't work. Don't use it.
    """

    return False

    # make empty 3D (n x 4 x 4) result array.
    # n tracks, 4 reco hits per track (all others are discarded),
    # 4 vales per reco (sector id, x, y, z)
    resultTracks = np.empty((0, 4, 4))

    runIndex = 0
    for arrayDict in uproot.iterate(
        filename,
        [
            "LMDHitsMerged.fSensorID",
            "LMDHitsMerged.fX",
            "LMDHitsMerged.fY",
            "LMDHitsMerged.fZ",
        ],
        library="np",
        allow_missing=True,  # some files may be empty, skip those
    ):
        ids = arrayDict["LMDHitsMerged.fSensorID"]
        recoX = arrayDict["LMDHitsMerged.fX"]
        recoY = arrayDict["LMDHitsMerged.fY"]
        recoZ = arrayDict["LMDHitsMerged.fZ"]

        # create mask for events that have exactly 4 hits
        mask = [event.size == 4 for event in ids]

        # make a real 2D array from array[array[ ]]
        maskedIDs = np.stack(ids[mask])

        # calculate module number from sensor IDs
        # use look up table for that
        # thank you TheodrosZelleke for this insanely smart idea
        # https://stackoverflow.com/a/14448935
        sectorIDfromLut = sectorIDlut[maskedIDs]

        # make arrays of arrays to 2d arrays
        xFlat = np.stack(recoX[mask])
        yFlat = np.stack(recoY[mask])
        zFlat = np.stack(recoZ[mask])

        # now, some recos are at -10000 or something in X/Y. Fuck those.
        xMask = np.abs(xFlat) < 100
        yMask = np.abs(yFlat) < 100

        allMaskX = np.array([all(x) for x in xMask])
        allMaskY = np.array([all(x) for x in yMask])

        allMask = allMaskX & allMaskY

        # transpose to assemble and transpose again
        theseTracks = np.array([sectorIDfromLut[allMask].T, xFlat[allMask].T, yFlat[allMask].T, zFlat[allMask].T]).T
        # theseTracks = np.array([sectorIDfromLut.T, xFlat.T, yFlat.T, zFlat.T]).T

        # sort every 4-hit combo by z value. do this for every event.
        # this is REQUIRED for the aligner
        # I know it looks weird seeing a list comprehension here instead of something
        # more NumPythonic, but this seems to be fastest after all
        # the numpy version would probably look like this:
        # sortedResultArray = theseTracks[:,theseTracks[:,:,3].argsort()]
        # or
        # sortedResultArray = np.einsum('iijk->ijk', theseTracks[:,theseTracks[:,:,0].argsort()])
        # and be 20x slower (and require MUCH more memory)
        sortedResultArray = np.array(
            [subarray[subarray[:, 3].argsort()] for subarray in theseTracks]
        )

        # stack this files' content with the others
        resultTracks = np.vstack((resultTracks, sortedResultArray))

        runIndex += 1
        if runIndex == maxNoOfFiles:
            break

    return resultTracks


In [7]:
class  CorridorFitter:
    def __init__(self, tracks):
        self.tracks = tracks
        self.nTrks = len(self.tracks)
        self.useAnchor = False

    def useAnchorPoint(self, point):
        assert len(point) == 3 or len(point) == 4
        self.anchorPoint = point
        self.useAnchor = True

    def fitTracksSVD(self):

        self.fittedTrackArr = np.zeros((self.nTrks, 2, 3))

        for i in range(self.nTrks):
            # cut fourth entry, sometimes this is the sensorID or homogeneous coordinate
            trackRecos = self.tracks[i][:, :3]

            if self.useAnchor:
                trackRecos = np.vstack((self.anchorPoint[:3], trackRecos))

            # see https://stackoverflow.com/questions/2298390/fitting-a-line-in-3d
            meanPoint = trackRecos.mean(axis=0)
            _, _, vv = np.linalg.svd(trackRecos - meanPoint)

            self.fittedTrackArr[i][0] = meanPoint

            # flip tracks that are fitted backwards
            if vv[0, 2] < 0:
                self.fittedTrackArr[i][1] = -vv[0]
            else:
                self.fittedTrackArr[i][1] = vv[0]

        return self.fittedTrackArr

In [8]:
# ? cuts on track x,y direction
def dynamicTrackCut(newTracks, cutPercent=2):
    com = np.average(newTracks[:, 1, :3], axis=0)

    # shift newhit2 by com of differences
    newhit2 = newTracks[:, 1, :3] - com
    newDist = np.power(newhit2[:, 0], 2) + np.power(newhit2[:, 1], 2)

    cut = int(len(newhit2) * (cutPercent / 100.0))
    newTracks = newTracks[newDist.argsort()]
    newTracks = newTracks[:-cut]
    return newTracks

# ? cuts on reco-track distance
def dynamicRecoTrackDistanceCut(newTracks, cutPercent=2):

    # don't worry, numpy arrays are copied by reference
    tempTracks = newTracks

    for i in range(4):
        trackPosArr = tempTracks[:, 0, :3]
        trackDirArr = tempTracks[:, 1, :3]
        recoPosArr = tempTracks[:, 2 + i, :3]

        # norm momentum vectors, this is important for the distance formula!
        trackDirArr = (
            trackDirArr / np.linalg.norm(trackDirArr, axis=1)[np.newaxis].T
        )

        # vectorized distance calculation
        tempV1 = trackPosArr - recoPosArr
        tempV2 = (tempV1 * trackDirArr).sum(axis=1)
        dVec = tempV1 - tempV2[np.newaxis].T * trackDirArr
        dVec = dVec[:, :2]
        newDist = np.power(dVec[:, 0], 2) + np.power(dVec[:, 1], 2)

        # cut
        cut = int(len(dVec) * (cutPercent / 100.0))
        tempTracks = tempTracks[newDist.argsort()]
        tempTracks = tempTracks[:-cut]

    return tempTracks

In [9]:
def best_fit_transform(A, B):
    """
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    """

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
        Vt[m - 1, :] *= -1
        R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R, centroid_A.T)

    # homogeneous transformation
    T = np.identity(m + 1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t

def getMatrix(trackPositions, recoPositions, use2D=False):

    # use 2D, use only in LMD local!
    if use2D:
        T, _, _ = best_fit_transform(
            trackPositions[..., :2], recoPositions[..., :2]
        )

        # homogenize
        resultMat = np.identity(4)
        resultMat[:2, :2] = T[:2, :2]
        resultMat[:2, 3] = T[:2, 2]
        return resultMat

    else:
        T, _, _ = best_fit_transform(trackPositions, recoPositions)
        return T


In [10]:
def alignSectorICP(sectorRecos, sector, maxNoTrks=40000):
    """
    @param sectorRecos np array for track positon, direction and recos
    """

    iterations = 5

    # TODO: add to config!
    preTransform = True

    detectorMatrices = loadMatrices("../config/detectorMatricesIdeal.json")
    avgMisalignments = loadMatrices("../matrices/100u-case-1/externalMatrices-modules.json")


    #! anchor points are loaded here!
    with open(anchorPointFile) as f:
        anchorPoints = json.load(f)

        # check if this is the new version of the anchorpoint format
        if 'version' in anchorPoints:
            if anchorPoints['version'] == 1:
                # if yes, make every entry a homogeneous point
                for key in anchorPoints:
                    # well except the version string of course
                    if key != 'version':
                        anchorPoints[key] = [0,0,anchorPoints[key],1]
            
        # the old version already contains homogeneous points
        else:
            pass

    with open("../config/sectorPaths.json") as f:
        allModulePaths = json.load(f)

    # get relevant module paths
    modulePaths = allModulePaths[str(sector)]

    # make 4x4 matrices to module positions
    moduleMatrices = np.zeros((4, 4, 4))
    for i in range(len(modulePaths)):
        path = modulePaths[i]
        moduleMatrices[i] = np.array(detectorMatrices[path])

    # * use average misalignment
    averageShift = avgMisalignments[str(sector)]

    # assing given tracks to internal variable
    newTracks = sectorRecos

    # use only N tracks:
    if maxNoTrks > 0:
        newTracks = newTracks[:maxNoTrks]

    sectorString = str(sector)
    # transform all recos to LMD local
    if preTransform:
        matToLMD = np.linalg.inv(np.array(detectorMatrices["/cave_1/lmd_root_0"]))
        for i in range(4):
            newTracks[:, i + 2] = (matToLMD @ newTracks[:, i + 2].T).T

    # transform anchorPoints to PANDA global
    else:
        matFromLMD = np.array(detectorMatrices["/cave_1/lmd_root_0"])
        anchorPoints[sectorString] = matFromLMD @ anchorPoints[sectorString]

    print(f"==================================================")
    print(f"        module aligner for sector {sector}")
    print(f"==================================================")

    print(f"number of tracks: {len(newTracks)}")
    print(f"anchor point: {anchorPoints[sectorString]}")

    # do a first track fit, otherwise we have no starting tracks
    recos = newTracks[:, 2:6]
    corrFitter = CorridorFitter(recos)
    corrFitter.useAnchorPoint(anchorPoints[sectorString][:3])
    resultTracks = corrFitter.fitTracksSVD()

    # update current tracks
    newTracks[:, 0, :3] = resultTracks[:, 0]
    newTracks[:, 1, :3] = resultTracks[:, 1]

    # prepare total matrices
    totalMatrices = np.zeros((4, 4, 4))
    for i in range(4):
        totalMatrices[i] = np.identity(4)

    # * =========== iterate cuts and calculation
    for iIteration in range(iterations):
        # print(f"running iteration {iIteration}, {len(newTracks)} tracks remaining...")

        newTracks = dynamicRecoTrackDistanceCut(newTracks)
        # 4 planes per sector
        for i in range(4):
            trackPosArr = newTracks[:, 0, :3]
            trackDirArr = newTracks[:, 1, :3]
            recoPosArr = newTracks[:, 2 + i, :3]

            # norm momentum vectors, this is important for the distance formula!
            trackDirArr = (
                trackDirArr / np.linalg.norm(trackDirArr, axis=1)[np.newaxis].T
            )

            # vectorized distance calculation
            tempV1 = trackPosArr - recoPosArr
            tempV2 = (tempV1 * trackDirArr).sum(axis=1)
            dVec = tempV1 - tempV2[np.newaxis].T * trackDirArr

            # the vector thisReco+dVec now points from the reco hit to the intersection of the track and the sensor
            pIntersection = recoPosArr + dVec

            # we want FROM tracks TO recos
            T0inv = getMatrix(recoPosArr, pIntersection, preTransform)
            totalMatrices[i] = T0inv @ totalMatrices[i]

            # transform recos, MAKE SURE THEY ARE SORTED
            newTracks[:, i + 2] = (T0inv @ newTracks[:, i + 2].T).T

        # direction cut again
        if iIteration < 3:
            newTracks = dynamicTrackCut(newTracks, 1)

        # do track fit
        corrFitter = CorridorFitter(newTracks[:, 2:6])
        resultTracks = corrFitter.fitTracksSVD()

        # update current tracks
        newTracks[:, 0, :3] = resultTracks[:, 0]
        newTracks[:, 1, :3] = resultTracks[:, 1]

    # * =========== store matrices
    # 4 planes per sector

    result = {}

    # transform the alignment matrices back INTO the system of their respective module
    # since that's where FAIRROOT applies them
    for i in range(4):
        # ideal module matrices!
        toModMat = moduleMatrices[i]

        if preTransform:
            totalMatrices[i] = baseTransform(totalMatrices[i], np.linalg.inv(matToLMD))
            totalMatrices[i] = baseTransform(totalMatrices[i], np.linalg.inv(toModMat))
        else:
            totalMatrices[i] = baseTransform(totalMatrices[i], np.linalg.inv(toModMat))

        # add average shift (external measurement)
        totalMatrices[i] = totalMatrices[i] @ averageShift
        result[modulePaths[i]] = totalMatrices[i]

    print(f"-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    print(f"        module aligner for sector {sector} done!         ")
    print(f"-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    return result


In [11]:
# read 350 MB of csv like it's nothing
resultArray = readRecoHitsFromCSVFile(recoFile)

In [12]:
# actually run the aligner!
alignmentMatices = {}

# translate the reco hits to format for module Aligner
for i in range(10):
    # apply mask so only one sector is chosen
    # careful, this only checks if the first hit
    # is in the correct sector!
    sectorMask = resultArray[:, 0, 0] == i

    # create new empty array thay has the required structure
    sectorTracksXYZ = resultArray[sectorMask][:, :, 1:4]
    nTrks = len(sectorTracksXYZ)

    # assign recos from input array to new array for this sector
    trackVectorForAligner = np.ones((nTrks, 6, 4))
    trackVectorForAligner[:, 2:6, :3] = sectorTracksXYZ[:, 0:4]

    alignmentMatices |= alignSectorICP(trackVectorForAligner, i)


        module aligner for sector 0
number of tracks: 40000
anchor point: [0, 0, -1105.0534856598595, 1]
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        module aligner for sector 0 done!         
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        module aligner for sector 1
number of tracks: 40000
anchor point: [0, 0, -1104.9551320665605, 1]
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        module aligner for sector 1 done!         
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        module aligner for sector 2
number of tracks: 40000
anchor point: [0, 0, -1111.6332939639292, 1]
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        module aligner for sector 2 done!         
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        module aligner for sector 3
number of tracks: 40000
anchor point: [0, 0, -1116.9724383008852, 1]
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
        module aligne

In [13]:
# save alignment matrix!
saveMatrices(alignmentMatices, alignmentMatrixFilename)

In [14]:
paths = []

for iHalf in range(2):
    for iPlane in range(4):
        for iModule in range(5):
            paths.append(f'/cave_1/lmd_root_0/half_{iHalf}/plane_{iPlane}/module_{iModule}')

# oldAnchorMatrices = loadMatrices('../../output/alMat-modules-oldAnchors.json')
newAnchorMatrices = loadMatrices(alignmentMatrixFilename)
oldAnchorMatrices = loadMatrices('../output/alMat-modules-newAnchors.json')
mActual = loadMatrices('../matrices/100u-case-1/misMat-modules.json')

for path in paths:

    print('--------------------------------------------------------:')
    print('Path: ' + path)
    print('Difference old:\n')
    print((np.linalg.inv(oldAnchorMatrices[path]) @ mActual[path])*1e4)
    print('Difference new:\n')
    print((np.linalg.inv(newAnchorMatrices[path]) @ mActual[path])*1e4)


--------------------------------------------------------:
Path: /cave_1/lmd_root_0/half_0/plane_0/module_0
Difference old:

[[ 9999.999954     0.952684    -0.073566    13.707804]
 [   -0.952683  9999.999953     0.201274   -13.47228 ]
 [    0.073585    -0.201267  9999.999998    -0.056145]
 [    0.           0.           0.       10000.      ]]
Difference new:

[[10000.           0.065203     0.           1.782422]
 [   -0.065203 10000.           0.          -4.727228]
 [   -0.          -0.       10000.           0.      ]
 [    0.           0.           0.       10000.      ]]
--------------------------------------------------------:
Path: /cave_1/lmd_root_0/half_0/plane_0/module_1
Difference old:

[[ 9999.999884    -1.521342    -0.045777     0.155953]
 [    1.521341  9999.999883    -0.143047    11.410553]
 [    0.045799     0.14304   9999.999999     0.658199]
 [    0.           0.           0.       10000.      ]]
Difference new:

[[ 9999.999997    -0.246516     0.          -2.465256]
